In [1]:
import os
import numpy as np
import pandas as pd
from tensorflow.keras.utils import to_categorical

def load_file(filepath):
	dataframe = pd.read_csv(filepath, header=None, skiprows=1)
	return dataframe.values

PNES_path ="../pilot/PNES/" # where the folders are saved
ES_path ="../pilot/ES/"

length = float('inf')
PNESdata = []
ESdata = []

In [2]:
pose_path = os.path.join(PNES_path, 'OpenPose Pose')
face_path = os.path.join(PNES_path, 'OpenFace Normalised')
for index in os.listdir(pose_path):
    pose_csv = os.path.join(pose_path, index)
    face_csv = os.path.join(face_path, index)
    if pose_csv.endswith(".csv") or pose_csv.endswith(".CSV"):    
        if face_csv.endswith(".csv") or face_csv.endswith(".CSV"):
            pose_values = load_file(pose_csv)
            face_values = load_file(face_csv)        
            short = min(pose_values.shape[0],face_values.shape[0])
            dataframe = np.concatenate((pose_values[:short,:],face_values[:short,3:]),1)
            PNESdata.append(dataframe)
            if short < length:
                length = short
                print(length)

3440
3400


In [3]:
pose_path = os.path.join(ES_path, 'OpenPose Pose')
face_path = os.path.join(ES_path, 'OpenFace Normalised')
for index in os.listdir(pose_path):
    pose_csv = os.path.join(pose_path, index)
    face_csv = os.path.join(face_path, index)
    if pose_csv.endswith(".csv") or pose_csv.endswith(".CSV"):    
        if face_csv.endswith(".csv") or face_csv.endswith(".CSV"):
            pose_values = load_file(pose_csv)
            face_values = load_file(face_csv)        
            short = min(pose_values.shape[0],face_values.shape[0])
            dataframe = np.concatenate((pose_values[:short,:],face_values[:short,3:]),1)
            ESdata.append(dataframe)
            if short < length:
                length = short
                print(length)

3280
2269


In [4]:
for idx, array in enumerate(PNESdata):
    PNESdata[idx] = array[:length,::]
    
PNESarray = np.array(PNESdata)
PNESlabel = np.reshape(np.zeros(PNESarray.shape[0]),[PNESarray.shape[0],1])

for idx, array in enumerate(ESdata):
    ESdata[idx] = array[:length,::]
    
ESarray = np.array(ESdata)
ESlabel = np.reshape(np.ones(ESarray.shape[0]),[ESarray.shape[0],1])

X = np.concatenate((PNESarray,ESarray),axis=0)
y = np.concatenate((PNESlabel,ESlabel),axis=0)
y = to_categorical(y)

In [5]:
print(PNESarray.shape)
print(ESarray.shape)
print(PNESlabel.shape)
print(ESlabel.shape)
print(X.shape)
print(y.shape)

(11, 2269, 786)
(16, 2269, 786)
(11, 1)
(16, 1)
(27, 2269, 786)
(27, 2)


In [6]:
outfile = "../Datasets/Pilot_Pose+Face_Video.npz"
np.savez(outfile, X, y)